# 제안요청서 요약 서비스

기업/정부 제안요청서 내용을 NLP/LLM을 활용하여 요약하는 노트북입니다.
GCP VM JupyterLab에서 실행할 수 있습니다.

## 1. 환경 설정 및 패키지 설치

In [ ]:
# 필요한 패키지 설치 (최초 1회)
# !pip install openai python-docx PyPDF2 python-dotenv

In [ ]:
import os
from pathlib import Path
from dataclasses import dataclass

In [ ]:
# 경로 설정 (JupyterLab 시작 위치에 맞게 자동 조정)
_cwd = Path.cwd()
BASE_DIR = _cwd if (_cwd / "notebooks").exists() else _cwd.parent
DATA_DIR = BASE_DIR / "data" / "raw"
DATA_DIR.mkdir(parents=True, exist_ok=True)  # 문서 저장용 디렉토리 생성

# API 키 (환경변수 또는 직접 설정)
# os.environ["OPENAI_API_KEY"] = "your_api_key"

## 2. 문서 로더

In [ ]:
def load_docx(file_path: Path) -> str:
    """.docx 파일에서 텍스트 추출"""
    from docx import Document
    doc = Document(file_path)
    return "\n".join(para.text for para in doc.paragraphs)

In [ ]:
def load_pdf(file_path: Path) -> str:
    """.pdf 파일에서 텍스트 추출"""
    import PyPDF2
    text_parts = []
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text_parts.append(page.extract_text() or "")
    return "\n".join(text_parts)

In [ ]:
def load_document(file_path: str | Path) -> str:
    """문서 로드 - .docx, .pdf 지원"""
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")
    
    suffix = path.suffix.lower()
    if suffix == ".docx":
        return load_docx(path)
    elif suffix == ".pdf":
        return load_pdf(path)
    else:
        raise ValueError(f"지원하지 않는 형식: {suffix}. 지원: .docx, .pdf")

## 3. 요약기 (LLM)

In [ ]:
@dataclass
class SummaryResult:
    """요약 결과"""
    summary: str
    key_points: list
    original_length: int
    summary_length: int

In [ ]:
def summarize_proposal(text: str, model: str = "gpt-4o-mini") -> SummaryResult:
    """
    제안요청서 텍스트 요약
    TODO: OpenAI 등 LLM API 연동 구현
    """
    # from openai import OpenAI
    # client = OpenAI()
    # response = client.chat.completions.create(...)
    
    return SummaryResult(
        summary="[요약 구현 예정]",
        key_points=["핵심 포인트 1", "핵심 포인트 2"],
        original_length=len(text),
        summary_length=0,
    )

## 4. 전체 파이프라인 실행

In [ ]:
# 문서 경로 지정 (data/raw/ 아래에 .docx 또는 .pdf 배치)
document_path = DATA_DIR / "sample.docx"  # 또는 "sample.pdf"

# 파일이 있을 때만 실행
if document_path.exists():
    text = load_document(document_path)
    result = summarize_proposal(text)
    print(f"원본 길이: {result.original_length}자")
    print(f"요약: {result.summary}")
    print(f"핵심 포인트: {result.key_points}")
else:
    print(f"샘플 문서를 {document_path} 에 배치해주세요.")
    print("문서 로드 및 요약 파이프라인 뼈대가 준비되었습니다.")